# `AzureAIFaceAnalysisTool`

## Overview

### Integration details

>The `AzureAIFaceAnalysisTool` acts as a wrapper around the Azure AI Face Service/API. The Tool will detect faces is in an image 
>and can supply the position of faces, as well as important features like eyes, pupils, head pose etc.

### Tool features

This integration allows for the detection of faces using Azure's Face API. It supports various different models, but defaults to the latest. Furthermore, the user can configure which information they would like to be returned such as position of the faces and image quality of the faces.

## Setup

This section provides details about how the Azure AI Face integration works, including setup.

In [1]:
import os

from langchain import hub

We will use a prompt to instruct the model. LangChain prompts can be configured, but for simplicity, we will use a premade prompt from LangSmith. This requires an API key, which can be set up here after registration.

In [2]:
LANGSMITH_KEY = os.environ["LANGSMITH_KEY"]
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

Now we can use the `AzureAIFaceAnalysisTool` combined with a model, using `create_structured_chat_agent`.

In [5]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.azure_ai_services.face_analysis import (
    AzureAIFaceAnalysisTool,
)
from langchain_openai import AzureChatOpenAI

### Credentials

Credentials can be set by being passed as parameters and should be stored locally as environment variables.

In [6]:
face_endpoint = os.environ["AZURE_AI_FACE_ENDPOINT"]
face_key = os.environ["AZURE_AI_FACE_KEY"]

## Instantiation

Credentials can be passed directly, but they can also be retrieved automatically by the constructor if environment variables named `AZURE_AI_FACE_ENDPOINT` and `AZURE_AI_FACE_KEY` are set.

In [7]:
face = AzureAIFaceAnalysisTool(
    azure_ai_face_key=face_key,
    azure_ai_face_endpoint=face_endpoint,
)

In [8]:
tools = [face]

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["COMPLETIONS_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

Creating an `AgentExecutor` chain allows a model to use tools to assist in it's response.

In [9]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## Invocation

Get either a local file path or remote image url as input

In [21]:
image_path = "../../example_data/face.jpg"

In [22]:
input = image_path

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

Firstly, the tool can be used by directly passing input as an argument.

In [23]:
face.invoke({"query": input})

"FACE: 1\nfaceId: 9b31b927-24a5-405c-95a9-d76f457e7160\nfaceRectangle: {'top': 760, 'left': 1159, 'width': 853, 'height': 1337}\nfaceLandmarks: {'pupilLeft': {'x': 1293.2, 'y': 1319.9}, 'pupilRight': {'x': 1671.1, 'y': 1306.7}, 'noseTip': {'x': 1383.7, 'y': 1593.6}, 'mouthLeft': {'x': 1333.1, 'y': 1821.3}, 'mouthRight': {'x': 1614.4, 'y': 1823.6}, 'eyebrowLeftOuter': {'x': 1167.6, 'y': 1199.4}, 'eyebrowLeftInner': {'x': 1348.4, 'y': 1214.3}, 'eyeLeftOuter': {'x': 1237.2, 'y': 1321.2}, 'eyeLeftTop': {'x': 1294.7, 'y': 1286.6}, 'eyeLeftBottom': {'x': 1289.7, 'y': 1347.3}, 'eyeLeftInner': {'x': 1351.2, 'y': 1324.3}, 'eyebrowRightInner': {'x': 1522.6, 'y': 1201.8}, 'eyebrowRightOuter': {'x': 1826.9, 'y': 1196.0}, 'eyeRightInner': {'x': 1605.4, 'y': 1317.3}, 'eyeRightTop': {'x': 1662.8, 'y': 1268.2}, 'eyeRightBottom': {'x': 1672.7, 'y': 1335.6}, 'eyeRightOuter': {'x': 1743.3, 'y': 1305.7}, 'noseRootLeft': {'x': 1396.7, 'y': 1342.7}, 'noseRootRight': {'x': 1499.7, 'y': 1337.8}, 'noseLeftAlar

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

By using `.invoke`, the model can be told what to do and assess if using the tools it was given would assist in it's response. Here we will use a remote url as input.

In [10]:
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Guy_Sebastian_2014.jpg/330px-Guy_Sebastian_2014.jpg"
input = image_path

In [11]:
agent_executor.invoke(
    {"input": f"Can you check the following image for faces : {input}"}
)



> Entering new AgentExecutor chain...
{
  "action": "azure_ai_face_analysis",
  "action_input": {
    "query": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Guy_Sebastian_2014.jpg/330px-Guy_Sebastian_2014.jpg"
  }
}FACE: 1
faceId: ab842360-a7fc-4570-8082-8f0928064141
faceRectangle: {'top': 69, 'left': 47, 'width': 209, 'height': 266}
faceLandmarks: {'pupilLeft': {'x': 101.5, 'y': 192.0}, 'pupilRight': {'x': 194.8, 'y': 183.8}, 'noseTip': {'x': 148.4, 'y': 237.3}, 'mouthLeft': {'x': 109.1, 'y': 265.7}, 'mouthRight': {'x': 199.8, 'y': 257.2}, 'eyebrowLeftOuter': {'x': 70.6, 'y': 174.6}, 'eyebrowLeftInner': {'x': 120.7, 'y': 175.5}, 'eyeLeftOuter': {'x': 85.3, 'y': 192.7}, 'eyeLeftTop': {'x': 102.7, 'y': 188.3}, 'eyeLeftBottom': {'x': 100.1, 'y': 195.2}, 'eyeLeftInner': {'x': 118.0, 'y': 191.7}, 'eyebrowRightInner': {'x': 168.4, 'y': 172.0}, 'eyebrowRightOuter': {'x': 225.9, 'y': 164.1}, 'eyeRightInner': {'x': 178.8, 'y': 186.5}, 'eyeRightTop': {'x': 193.0, 'y': 179.8}, 'ey

{'input': 'Can you check the following image for faces : https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Guy_Sebastian_2014.jpg/330px-Guy_Sebastian_2014.jpg',
 'output': 'The image contains one face. The faceId is ab842360-a7fc-4570-8082-8f0928064141, and the face is located at the rectangle with top: 69, left: 47, width: 209, and height: 266. The face has various landmarks and attributes, including a high quality for recognition.'}

## Chaining

When creating an `AgentExecutor` as described earlier, an execution chain is formed. The sequence of events will be printed using the given prompt, and actions will occur in a chain-like manner.

In [26]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## API reference

[Azure AI Face python SDK](https://learn.microsoft.com/python/api/overview/azure/ai-vision-face-readme?view=azure-python-preview) | [Azure AI Face Overview](https://learn.microsoft.com/azure/ai-services/computer-vision/overview-identity)